In [ ]:
from getStats import *
from lcu_driver import Connector
import pickle
import sys
import os
import pandas as pd
import sklearn

In [ ]:
#Establishing the connector to the League Client Data and 
connector = Connector()
names = []

In [ ]:
@connector.ready
async def connect(connection):
    for i in range(5):
        summoner = await connection.request('get', '/lol-champ-select/v1/summoners/'+str(i))
        temp = await summoner.json()
        sumId = temp['summonerId']
        name = await connection.request('get', '/lol-summoner/v1/summoners/'+str(sumId))
        temp2 = await name.json()
        names.append(temp2['displayName'])
    userName = await connection.request('get', '/lol-summoner/v1/current-summoner')
    name = await userName.json()
    names.remove(name['displayName'])

In [ ]:
connector.start()
#print(names)

In [ ]:
#Building a dataframe for the users in the current lobby
df = pd.DataFrame(columns=['ign', 'avgKDA', 'avgVisionScore', 'avgWards', 'percentTop', 'avgCSD'])
for ign in names:
    accountID=getSummonerID(ign, api)
    matchIDs = getMatchData(accountID, api)
    row = getMatchStats(matchIDs, ign ,api, df)
    df = df.append(row, ignore_index=True)

In [ ]:
model_path = "" #Add model path here for whichever 
model = pickle.load(open(model_path, 'rb'))
X = df.drop(columns=['ign'])
preds = model.predict(X)

if preds.sum() < 3:
    os.system("TASKKILL /F /IM leagueclient.exe")